# 딥러닝 강의 - softmax classification과 cnn 

In [1]:
import tensorflow as tf
import random


from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
#tensorflow에서 기본적으로 제공하는 mnist데이터를 받아옵니다. 
#이 데이터는 사진(28*28*1)으로, 흰색 배경에 검은색으로 0~9까지 숫자가 필기체로 적혀있습니다. 
#이때 이 사진에 어떤 수가 적혀있는 것인지 맞추게 하는 것이 이번 딥러닝 예제의 목표입니다..!
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

NameError: name 'input_data' is not defined

# softmax classification-basic NN

### 텐서플로우 노드를 만들어봅시다.

In [ ]:

#이때 placeholder는 나중에 데이터를 넣을 수 있는 '통' 같은 개념이라 이해하시면 편합니다. 

x=tf.placeholder(tf.float32,[None,784],name='x') ##변수를 넣어두는곳, 784개가 들어감 - 픽셀 수!
y=tf.placeholder(tf.float32,[None,10],name='y')
    
    
    
#여기서 784라는 숫자는 사진이 28*28*1이기 때문에 한 사진당 총 픽셀이 28의 제곱인 784개 들어있기 떄문에 사용합니다. 
#즉 softmax classification 방법은 데이터를 일렬로 쭉 펴서 저장해두는 방식을 사용합니다.


### 가설함수 H(x)=Wx+b를 만들어 봅시다. 

In [ ]:
with tf.name_scope("layer1"): ##1층으로 묶음 - 다른 층/ 단계로 filter
    #우리가 글로 적을 때는 Wx+b라 적지만 실제 코딩을 할떄는 xW+b와 같은 형태입니다.
    #x가 n*784, W가 784*28 b가 n*28이고 따라서 layer1은 n*28이 될 것입니다. 
    W1=tf.Variable(tf.random_normal([784,28]),name='weight1') ##784는 인풋의 개수, 28은 아웃풋의 개수 - W는 웨이트
    b1=tf.Variable(tf.random_normal([28]),name='bias1') ## bias는 아웃풋 개수만큼만 필요함
    layer1=tf.sigmoid(tf.matmul(x,W1)+b1) # 시그모이드로  logit 만듦! sigmoid
                                                # 이유 - 0과 1 사이면 확률로 변환 가능함 - 0.5를 기준으로 나누어서 classification 용이
    
    
    #summary.histogram은 tensorboard에서 표를 그릴때 사용하는 것으로 
    #나중에 tensorboard를 배우고 싶으신 분들이 있을까봐 참고용으로 지우지 않고 넣어두었습니다. 
    #w1_hist=tf.summary.histogram("weighth1",W1)
    #b1_hist=tf.summary.histogram("biash1",b1)
    #layer1_hist=tf.summary.histogram("layer1",layer1)

with tf.name_scope("layer2"): #두번의 레이어로 게싼하면 더 편함
    W2=tf.Variable(tf.random_normal([28,10]),name='weight2')
    b2=tf.Variable(tf.random_normal([10]),name='bias2')
    
    #logits는 Wx+b로 우리가 W,b를 조절하며 계산,예측한 값을 의미합니다. 
    logits=tf.matmul(layer1,W2)+b2
    hypo=tf.nn.softmax(logits) # 다 더해서 1이 나오게 softmax - 이해하기 쉽게 (layer 1의 값을 layer 2에 사용, 한번 더 filter)
   
    #w2_hist=tf.summary.histogram("weighth2",W2)
    #b2_hist=tf.summary.histogram("biash2",b2)
    #logits_hist=tf.summary.histogram("logits",logits)
    

### cost함수를 만들어 봅시다

In [ ]:
# cost/ penalty를 계산하는 법 만들어서 이후 모델에 적용, 0 혹은 1의 target에 최대한 맞을 수 있도록 조정하는 법 - 틀리면 마이너스를 주어 
# 예측이 틀렸음을 알림
# 

with tf.name_scope("cost"):
    cost=tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=y)
    #cost_sum=tf.summary.scalar("cost",cost)

### cost를 줄이는 optimizer로 우리가 잘 아는 경사하강법(gradient descent)을 사용합니다.

In [ ]:
# 경사하강법 - smooth하게 내려가는  cost function을 만들었으니 이에 맞게 최저값을 찾기

with tf.name_scope("train"):
    optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

### 우리가 만든 가설함수의 정확성을 확인하기 위한 코드입니다.

In [ ]:

prediction=tf.argmax(hypo,axis=1)

is_correct=tf.equal(prediction,tf.argmax(y,1))
#여기서 주의할 점은 뒤에 실행 코드를 보시면 아시겠지만 이 x,y에는 train이 아닌 test데이터를 집어넣습니다.  
    # 정답이 맞았는지를 확인함 - 예측 0, 결과 0이면 1, 반대면 0. 
    
accuracy=tf.reduce_mean(tf.cast(is_correct,tf.float32)) #맞은 수 / 전체 수로 얼마나 모델이 정확했는지 점수내기
#accuracy_sum=tf.summary.scalar("accuracy",accuracy)

# 각 행의 최대 값을 뽑아내기

### 지금까지는 가설함수와 층을 만들고, 코스트를 줄이는 방식(여기서는 경사하강법)을 정하는 등 텐서플로우 상에서 'graph를 그린 것'입니다. 
### 이제부터 할 것은 실제로 그 안에서 가설함수의 코스트를 줄이는 방향으로 학습을 진행하라는 코드를 볼 것입니다.  

In [ ]:
 
with tf.Session() as sess: #텐서플로우를 만들어서 이 틀에서 계산
    #이는 sess=tf.Session()과 같은 의미로 계산 과정을 시작하자는 시동을 거는 것입니다. 
    
    #parameters로 iter_epoch는 전체 학습 반복 횟수, batch_size는 한번에 읽어드려서 학습시키는 양을 의미합니다. 
    #batch_size는 왜 필요할까요? ## 처음 만든 node placeholder에 들어갈 파일의 크기
    iter_epoch=15 #학습 15번 반복
    batch_size=100 #placeholder에 한번에 몇개 파일 넣을지
    
    
    #global_step=0
    #merged=tf.summary.merge([accuracy_sum,cost_sum])
    #writer=tf.summary.FileWriter('c:\\GH\\tensor')
    #writer.add_graph(sess.graph)
    #valid_x=mnist.validation.images
    #valid_y=mnist.validation.labels
    
    
    #전역 변수 initializer을 사용하여 우리가 만든 W등의 초기화를 미리 지정한 방식(이 경우 random_normal)으로 초기화해줍니다. 
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(iter_epoch):
        avg_cost=0
        total_batch=int(mnist.train.num_examples/batch_size)
        #total_batch는 전체 train 데이터 개수를 미리 설정한 batch_size(이 경우 100)으로 나눈 값으로 
        #전체 train을 1번 완료 하려면 batch를 100개씩 넣는 작업을 몇번 반복해야하는 것인지를 뜻합니다 
        
        for i in range(total_batch):
            
            batch_x,batch_y=mnist.train.next_batch(batch_size)
            #train.next_batch는 다음 batch만큼의 데이터를 가져와서 batch_x,batch_y에 넣습니다.
            #이때 물론 batch_y는 각각의 사진의 실제 답,label입니다. 
            
            
            c,s,_=sess.run([cost,accuracy,optimizer],feed_dict={x:batch_x,y:batch_y})
            #cost, accuracy_sum에 batch만큼의 데이터를 집어넣어 줍니다. 이때 동시에 optimzer에도 넣어 cost를 줄여주는 학습을 진행합니다.
            #optimzer가 나타내는 값 자체는 중요하지 않기 때문에 _ 변수에 저장해주고 이런 _변수는 보통 활용하지 않을 변수에 사용합니다. 
            
            
            avg_cost+=c/total_batch
            #평균 비용은 각 cost를 반복횟수인 train_batch로 나누어서 구합니다.
            
            
            #s=sess.run(merged,feed_dict={x:valid_x,y:valid_y})  
            #writer.add_summary(s,global_step)  
            #global_step+=1
      
       # print('Epoch:','%d' %(epoch+1), 'cost=','{0}'.format(avg_cost))
        print("Accuracy",accuracy.eval(session=sess,feed_dict={x:mnist.test.images,y:mnist.test.labels}))
        
        r = random.randint(0, mnist.test.num_examples - 1)
        print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
        print("Prediction: ", sess.run(
            tf.argmax(logits, 1), feed_dict={x: mnist.test.images[r:r + 1]}))

        plt.imshow(mnist.test.images[r:r + 1].
                   reshape(28, 28), cmap='Greys', interpolation='nearest')
        plt.show()

# CNN

In [ ]:
tf.set_random_seed(777)  # reproducibility를 위해 지정해둡니다. 

#CNN이란 - 필터로 큰그림 쪼개보기
#mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100


In [ ]:
# dropout (keep_prob) rate  0.7~0.5 가 train시 권장되고 test 시에는 1을 사용해야합니다. 이 부분은 뒤에서 ppt와 함께 다시 설명해드리겠습니다. 
keep_prob = tf.placeholder(tf.float32)

### CNN 기본 노드들을 만들어봅시다

In [ ]:
#softmax 코드 부분과 다른점이 뭘까요..!?
    # 모형을 28 x 28 모양으로 바꿈 - 784개 픽셀을 28 x 28 어레이로 만들어서 이미지의 모습과 같게 만들기
    # softmax와 달리 쭉 펴서 분석을 안함
    
X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1])   # img 28x28x1 (흑백 사진이기에, 컬러였으면 RGB로 28*28*3 이었을 것입니다)
Y = tf.placeholder(tf.float32, [None, 10])

#X_img추가 - CNN은 픽셀을 쭉 일열로 받아들였으면 얜 사진의 모양을 그대로 받아들임! 처음에 -1인 이유는 몇개인지 모르니까

### cnn 필터들과 다층 layer를 만들어봅시다

In [ ]:
# L1 ImgIn shape=(?, 28, 28, 1)
#필터의 개별 크기는 [3,3,1(이는 흑백이어서,컬러면 3)]인 것이고 필터의 총 개수가 32개인것입니다. 
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
#    Conv     -> (?, 28, 28, 32)
#    Pool     -> (?, 14, 14, 32)
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME') #처음, 마지막은 다 1로 통일 #사이 2개는 움직이는 간격 - 하나씩 이동
                                                                #padding = 모서리란걸 알려줌
L1 = tf.nn.relu(L1) #1이상은 그대로, 1 이하는 다 0으로 통일 - CNN layer증가에 따라 값이 너무 작아지는걸 방지 (sigmoid는 최대 1), 그 이상값
                    #도 사용 가능하게 relu 사용
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME') #각 프레임/ filter에서 제일 큰값만 뽑아냄
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)  #학습시에 임의로 값들을 끊음 - 30%를 임의로 제거 - overfitting방지용
'''
Tensor("Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)
Tensor("dropout/mul:0", shape=(?, 14, 14, 32), dtype=float32)
'''
#여러개를 만드는데 w초기값은 다르다
# L2 ImgIn shape=(?, 14, 14, 32)
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
#    Conv      ->(?, 14, 14, 64)
#    Pool      ->(?, 7, 7, 64)
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)
'''
Tensor("Conv2D_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("dropout_1/mul:0", shape=(?, 7, 7, 64), dtype=float32)
'''

# L3 ImgIn shape=(?, 7, 7, 64)
W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01))
#    Conv      ->(?, 7, 7, 128)
#    Pool      ->(?, 4, 4, 128)
#    Reshape   ->(?, 4 * 4 * 128) # Flatten them for FC
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME') #layer 3개
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[
                    1, 2, 2, 1], padding='SAME')
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
L3_flat = tf.reshape(L3, [-1, 128 * 4 * 4])
#엇 왜 애써 사진 모양처럼 했던 것을 다시 flat하게 펼까요? 
    #분석을 위해서 컴퓨터가 이해할 수 있는 모형으로 전환해야하니까......??? padding 등 위해 이미지 모양 필요했고 완료했으니 돌리기(?)
'''
Tensor("Conv2D_2:0", shape=(?, 7, 7, 128), dtype=float32)
Tensor("Relu_2:0", shape=(?, 7, 7, 128), dtype=float32)
Tensor("MaxPool_2:0", shape=(?, 4, 4, 128), dtype=float32)
Tensor("dropout_2/mul:0", shape=(?, 4, 4, 128), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 2048), dtype=float32)
'''

### cnn 끝 부분에서 위에서 배웠던 softmax을 연결하여 학습시킵니다

In [ ]:
# L4 FC 4x4x128 inputs -> 625 outputs
#random_normal을 쓰다가 Xavier_initializer을 쓰는 이유는 뭘까요? 
    #컴퓨터에 초기값 설정, 초기 방향 제시 
W4 = tf.get_variable("W4", shape=[128 * 4 * 4, 625],
                     initializer=tf.contrib.layers.xavier_initializer()) #xavier? 초기방향 제시
b4 = tf.Variable(tf.random_normal([625]))
L4 = tf.nn.relu(tf.matmul(L3_flat, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)
'''
Tensor("Relu_3:0", shape=(?, 625), dtype=float32)
Tensor("dropout_3/mul:0", shape=(?, 625), dtype=float32)
'''
# L5 Final FC 625 inputs -> 10 outputs
W5 = tf.get_variable("W5", shape=[625, 10],
                     initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
logits = tf.matmul(L4, W5) + b5
'''
Tensor("add_1:0", shape=(?, 10), dtype=float32)
'''
#여기서 중요한 것.! 결국 최종적으로 마지막 logits은 10개의 라벨(0~9까지의 수)
#각각으로 예상할 확률로 만들어졌다는 것. ex: N(데이터 개수) * [1 2 3 2 5 1 0 1 2 3]
#여기까지가 첫 공식 만드는것

In [ ]:

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
#AdamOptimizer는 왜 사용할까요 ?
    
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) #경사하강법 사용 안함 - adam이 더 빠름

### 본격적인 학습을 시작합니다

In [ ]:
# 이 부분은 학습하는데 시간이 오래 걸립니다. 

sess = tf.Session()
#sess = tf.Session()와  위에서 쓴 with tf.Session() as sess는 동일한 의미입니다. 단, with - as : 방법은 
#들여쓰기를 한 부분까지만 Session이 유지되고 그 후에는 Session이 닫힙니다. 
sess.run(tf.global_variables_initializer())

print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

In [ ]:

# 내 모델의 정확도를 측정해보고 그림을 그려서 확인해봅시다. 



correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1}))


r = random.randint(0, mnist.test.num_examples - 1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Prediction: ", sess.run(
    tf.argmax(logits, 1), feed_dict={X: mnist.test.images[r:r + 1], keep_prob: 1}))

plt.imshow(mnist.test.images[r:r + 1].
           reshape(28, 28), cmap='Greys', interpolation='nearest')
plt.show()

## 강의 5-1 Logistic classification - Sigmoid의 이유 및 사용법 - linear의 한계 극복, 0~1로 분산
## classification에 적합하게 data를 분산하는 목적

## 강의 5-2 Logistic Classification - cost function and gradient decent 
## 기존 cost function은 적용 못함 - sigmoid 에 cost 대입하면 구불구불한 2차함수가 되고
## global minimum 찾을 수 없음.

## y=0, 1일때로 분류한 새로운 cost function 사용, 여기에 gradient decent 적용하여 cost를 최소로 하는 지점 발견할 수 있음. 

## 강의 Lab 5 - 실제로 데이터를 sigmoid 함수를 사용해서 분류해봄
## place holder에 input, output개수에 맞는 값을 입력 하고 sigmoid함수에 적용하여 학습
## 결과를 보고 결과값, 0/1로 분류, 그리고 이게 실제 target과 일치하는지를 계산하여 accuracy
## 까지 계산해볼 수 있다.